In [1]:
import folium
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# Read the cities CSV file
cities = pd.read_csv("minard_chard/cities.csv")

# Read the temperature CSV file
temperatures = pd.read_csv("minard_chard/temperature.csv")

# Read the troops CSV file
troops = pd.read_csv("minard_chard/troops.csv")

# Merge the DataFrames together on the long column, using the left join type
df = cities.merge(temperatures, on='long', how='left')
df = df.merge(troops, on='long', how='left')

# Fill in the missing values with NaN
df = df.fillna(method='ffill')

# Create a geopandas dataframe from the city data
city_gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['long'], df['lat_x']))

# Create a colormap for the temperature column
cmap = plt.cm.hot
norm = mcolors.Normalize(min(df['temp']), max(df['temp']))

# Create a list of colors for each city
colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

# Create a folium map centered on the first city in the DataFrame
m = folium.Map(location=[cities['lat'][0], cities['long'][0]], zoom_start=6, tiles='OpenStreetMap')

# Add markers for each city to the map, colored by temperature
for index, row in city_gdf.iterrows():
    # Create a popup with the city information
    popup_text = f"City: {row['city']}<br>Latitude: {row['lat_x']}<br>Longitude: {row['long']}<br>Temperature: {row['temp']}°C<br>Number of Troops: {row['survivors']}<br>Date: {row['date']}"
    
    # Add a marker for the city to the map
    folium.CircleMarker(location=[row['lat_x'], row['long']], radius=5, color=None, fill=True, fill_color=colors[index % len(colors)], fill_opacity=0.7, popup=popup_text).add_to(m)
    
    # Add the city name as a label to the marker
    folium.Marker(location=[row['lat_x'], row['long']], radius=5, color=None, fill=True, fill_color=colors[index % len(colors)], fill_opacity=0.7, popup=popup_text).add_to(m)


    # Create a tooltip with the city, temperature, date, and troops information
    tooltip_text = f"City: {row['city']}<br>Temperature: {row['temp']}°C<br>Date: {row['date']}<br>Troops: {row['survivors']}"
    
    # Add a marker for the city to the map
    folium.Marker(location=[row['lat_x'], row['long']], icon=None, popup=row['city'], tooltip=tooltip_text, 
                  color=colors[index % len(colors)], fill_color=colors[index % len(colors)]).add_to(m)

# Display the map
m